# String distances




In [3]:
import nltk
import numpy as np
import pandas as pd
#import editdistance

In [4]:
# nltk.download('words')
words = nltk.corpus.words.words()

In [5]:
pd.DataFrame(words).to_csv("words_nltk.csv")

## Assuming same cost for all edits

In [6]:
def create_memoization_table(X,Y):

    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x+1,len_y+1), dtype=np.int32)
    
    for i in range(len(X)+1):
        for j in range(len(Y)+1):

            if i == 0:
                D[i][j] = j    

            elif j == 0:
                D[i][j] = i  

            elif X[i-1] == Y[j-1]: 
                D[i][j] = D[i-1][j-1]

            else:
                D[i][j] = 1+min(D[i][j-1],      # Insert 
                                D[i-1][j],      # Remove 
                                D[i-1][j-1])    # Replace 
    return D

In [7]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"
D = create_memoization_table(x,y)
D[-1,-1]

6

In [8]:
x = "Elliot"
y = "Elia"
D = create_memoization_table(x,y)
D[-1,-1]

3

In [9]:
x = "hi"
y = "hill"
D = create_memoization_table(x,y)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))


The distance between hi and hill is 2


##### Timing implementation

In [10]:
x = "EXPONENTIAL"
y = "POLYNOMIAL"

In [11]:
def edit_distance_fast(x,y):
    D = create_memoization_table(x,y)
    return D[-1,-1]

In [12]:
%%timeit
edit_distance_fast(x,y)

178 µs ± 2.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
%%timeit
nltk.edit_distance(x,y)

54.5 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
nltk.edit_distance(x,y) == edit_distance_fast(x,y)

True

### Different costs per operation

In [15]:
def memoization_table_weighted(X,Y):

    len_x = len(X)
    len_y = len(Y)
    D = np.zeros((len_x + 1, len_y + 1), dtype=np.int32)
    D[:,0] = range(len_x + 1)
    D[0,:] = range(len_y + 1)

    w_sub = 1
    w_del = 1
    w_ins = 1

    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            del_char = D[i-1,j] + w_del
            ins_char = D[i,j-1] + w_ins

            if X[i-1] == Y[j-1]:
                Z = 0
            else:
                Z = w_sub
            sub_char = D[i-1,j-1] + Z

            D[i,j] = min(del_char, ins_char, sub_char)

    return D

In [16]:
x = "Elliot"
y = "Elia"
D = memoization_table_weighted(x, y)
print("\nThe distance between {} and {} is {}".format(x,y,D[-1,-1]))


The distance between Elliot and Elia is 3


# Speeding up code

Simple example with cython

In [17]:
%reload_ext cython

In [18]:
def fib(n):
    a = 0.
    b = 1.
    for i in range(n):
        a, b = a + b, a
    return a

In [25]:
%%cython --a
cpdef float cy_fib(int n):
    cdef int i
    cdef float a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [26]:
fib(10)

55.0

In [27]:
cy_fib(10)

NameError: name 'cy_fib' is not defined

In [60]:
import timeit

n_times = 100000
t_fib = timeit.timeit("fib(10)", setup="from __main__ import fib",number=n_times)
t_cyfib = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_fib_unit = t_fib/n_times

t_cyfib      = timeit.timeit("cy_fib(10)", setup="from __main__ import cy_fib",number=n_times)
t_cyfib_unit = t_cyfib/n_times

print(" Python version took: {} sec\n Cython version took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib, t_cyfib, t_fib/t_cyfib))

print("\n Python version 1 run took: {} sec\n Cython version took: {} sec\n Cython is {:.0f}x faster"\
      .format(t_fib_unit, t_cyfib_unit, t_fib_unit/t_cyfib_unit))

ImportError: cannot import name 'cy_fib' from '__main__' (unknown location)

### Speeding up edit distance



##### Exercise fill in cy_create_memoization_table so that it returns the matrix filled to compute the edit distance

In [30]:
%%cython --a

cimport numpy as np
import numpy as np
cimport cython

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
cpdef cy_create_memoization_table(str X, str Y):

    cdef int i, j, del_char, ins_char, sub_char, Z
    cdef int len_x = len(X)
    cdef int len_y = len(Y)
    cdef int [:, :] D =  np.zeros((len_x + 1, len_y + 1), dtype=np.int32)
    
    for i in range(len_x+1):
        D[i,0] = i

    for j in range(len_y+1):
        D[0,j] = j

    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            del_char = D[i-1,j] + 1
            ins_char = D[i,j-1] + 1

            if X[i-1] == Y[j-1]:
                Z = 0
            else:
                Z = 1
            sub_char = D[i-1,j-1] + Z

            D[i,j] = min(del_char, ins_char, sub_char)
    
    return D

In [31]:
D1 = create_memoization_table(x,y)
D1

array([[0, 1, 2, 3, 4],
       [1, 0, 1, 2, 3],
       [2, 1, 0, 1, 2],
       [3, 2, 1, 1, 2],
       [4, 3, 2, 1, 2],
       [5, 4, 3, 2, 2],
       [6, 5, 4, 3, 3]], dtype=int32)

In [32]:
D2 = cy_create_memoization_table(x,y)
D2 = np.asarray(D2)

NameError: name 'cy_create_memoization_table' is not defined

In [33]:
D2

NameError: name 'D2' is not defined

In [34]:
t_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; create_memoization_table(x,y)",
                                           setup="import numpy as np; from __main__ import create_memoization_table",
                                          number=5000)

NameError: name 'timeit' is not defined

In [89]:
t_cy_create_memoization_table = timeit.timeit("x='exponential'; y='polynomial'; cy_create_memoization_table(x,y)",
                                              setup="from __main__ import cy_create_memoization_table",
                                              number=5000)

In [90]:
t_nltk = timeit.timeit("x='exponential'; y='polynomial'; nltk.edit_distance(x,y)",
                        setup="import nltk ",
                        number=5000)

In [99]:
print(""" 
      Python version took: {} sec
      Cython version took: {} sec
      nltk   version took: {} sec
      Cython is {:.0f}x faster than python
      Cython is {:.0f}x faster than nltk
      """\
      .format(t_create_memoization_table, 
              t_cy_create_memoization_table,
              t_nltk, 
              t_create_memoization_table/t_cy_create_memoization_table,
              t_nltk/t_cy_create_memoization_table))


 
      Python version took: 1.3459972919999927 sec
      Cython version took: 0.006109708999929353 sec
      nltk   version took: 0.4242933330000369 sec
      Cython is 220x faster than python
      Cython is 69x faster than nltk
      


In [100]:
def edit_distance(x,y):
    return cy_create_memoization_table(x,y)[-1,-1]

### Return to the experiment where we computed closest word


##### Exercise: Return the last component of the DynamicProgramming matrix containing the edit distance

In [96]:
import editdistance

In [93]:
words = nltk.corpus.words.words()
len(words)

236736

In [94]:
%%time
mistake = "drauing" 
distances = []
for word in words:
    ed = nltk.edit_distance(mistake, word)
    distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(distances)])


the closest word is drawing
CPU times: user 13.2 s, sys: 19.7 ms, total: 13.3 s
Wall time: 13.3 s


In [97]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = editdistance.eval(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: user 343 ms, sys: 2.96 ms, total: 345 ms
Wall time: 345 ms


In [101]:
%%time
mistake = "drauing" 
cy_distances = []
for word in words:
    ed = edit_distance(mistake, word)
    cy_distances.append(ed)
    
print("\nthe closest word is", words[np.argmin(cy_distances)])


the closest word is drawing
CPU times: user 374 ms, sys: 2.7 ms, total: 377 ms
Wall time: 376 ms


In [102]:
distances == cy_distances

True

In [103]:
editdistance.eval("hi", "hi"), edit_distance("hi","hi")

(0, 0)

In [104]:
editdistance.eval("hi", "ho"), edit_distance("hi","ho")

(1, 1)



##### Interesting material on string similarities

Approximate string matching:

https://medium.com/@wolfgarbe/fast-approximate-string-matching-with-large-edit-distances-in-big-data-2015-9174a0968c0b

Levenshtein distance using a trie:

http://stevehanov.ca/blog/?id=114

About jaccard distance:

https://python.gotrained.com/nltk-edit-distance-jaccard-distance/


Nice work on string similarities:

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.19.7158&rep=rep1&type=pdf

